source of data: [The Japanese Mortality Database](https://www.ipss.go.jp/p-toukei/JMD/index-en.asp) > "Life expectancy at birth" tables<br>
wiki > [List of Japanese prefectures by life expectancy](https://en.wikipedia.org/wiki/List_of_Japanese_prefectures_by_life_expectancy) / [Продолжительность жизни в префектурах Японии](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_префектурах_Японии)<br />
see also: [Nippon.com](https://www.nippon.com/en/search.html?s=life%20expectancy)

In [2]:
import pandas as pd
import math
import re

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [3]:
pd.options.display.max_columns = 30
YEARS_SEL = list(range(2010, 2024))
SORTED_BY = [2020, 2023]

In [4]:
pattern_region_general = re.compile(r'(?<=^[\d.]{3}).*(?=(.ken|.to|.fu), Life expectancy)')  # what is in the most cases, like '47.Okinawa.ken'
pattern_region_1       = re.compile(r'(?<=^[\d.]{3}).*(?=, Life expectancy)')  # case of '01.Hokkaido' in the file Jp_01.csv
pattern_region_0       = re.compile(r'(?<=^).*(?=, Life expectancy)')  # case of 'Japan' in the file Jp_01.csv

def load_data_for_region(region_nmb):
    df = pd.read_csv(f"data-LE_at_birth/Jp_{region_nmb:02}.csv", sep='\t', skiprows=2, index_col='Year')

    with open(f"data-LE_at_birth/Jp_{region_nmb:02}.csv", 'r') as fh:
        title_line = fh.readline()
        if region_nmb >= 2:
            fragment_name_found = re.search(pattern_region_general, title_line)
        elif region_nmb == 1:
            fragment_name_found = re.search(pattern_region_1, title_line)
        elif region_nmb == 0:
            fragment_name_found = re.search(pattern_region_0, title_line)
        else:
            raise ValueError(f"Not expected region number: {region_nmb}")
        title = fragment_name_found.group() if fragment_name_found else 'region name is not found'

    df.name = title
        
    return df, title


df_t = df_m = df_f = pd.DataFrame()
ls_titles = []

for region_nmb in range(48):
    df_current, title_current = load_data_for_region(region_nmb)

    df_t = pd.concat([df_t, df_current.loc[YEARS_SEL, 'Total']], axis='columns')
    df_m = pd.concat([df_m, df_current.loc[YEARS_SEL, 'Male']], axis='columns')
    df_f = pd.concat([df_f, df_current.loc[YEARS_SEL, 'Female']], axis='columns')
    ls_titles.append(title_current)

df_t.columns = df_m.columns = df_f.columns = ls_titles

df_t = df_t.T
df_m = df_m.T
df_f = df_f.T

df_t.head(3)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Japan,82.95,82.71,83.21,83.43,83.68,83.92,84.10,84.22,84.32,84.46,84.71,84.58,84.10,84.14
Hokkaido,82.70,82.83,82.98,83.19,83.42,83.63,83.69,83.99,83.98,84.07,84.32,83.98,83.57,83.62
Aomori,81.33,81.55,81.50,81.91,82.05,82.41,82.65,82.74,82.81,82.60,83.09,82.96,82.69,82.22


In [5]:
# for region in df_t.sort_index().index.to_list():
#     print(f"    '{region}'\t: {{'jp': ('', ''), 'en': ('', ''), 'ru': ('', '')}},")

<br>

In [7]:
# sort values
df_t = pd.concat([df_t.loc[['Japan']],
                  df_t.drop(index='Japan').sort_values(by=SORTED_BY, ascending=False)])

ls_indexes_sorted = df_t.index.to_list()
df_m = df_m.loc[ls_indexes_sorted]
df_f = df_f.loc[ls_indexes_sorted]

df_t.head(3)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Japan,82.95,82.71,83.21,83.43,83.68,83.92,84.10,84.22,84.32,84.46,84.71,84.58,84.10,84.14
Shiga,83.66,83.84,83.98,84.20,84.52,84.63,84.99,84.96,85.15,85.35,85.71,85.60,85.09,85.19
Kyoto,83.50,83.54,83.62,83.88,84.18,84.49,84.74,84.89,85.07,85.20,85.54,85.39,84.64,84.89


<br>

In [9]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
mal.min_and_max_values(df_t, row_center=['Japan', 'Okinawa'], nmb=3, max_lng=9)

Number of records: 48


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
max,84.04 -Nagano,83.86 -Nagano,84.03 -Nagano,84.47 -Nagano,84.52 -Shiga,84.93 -Nagano,84.99 -Shiga,85.02 -Nagano,85.28 -Nagano,85.35 -Shiga,85.71 -Shiga,85.67 -Nagano,85.16 -Nagano,85.28 -Nagano
max_2,83.73 -Kumamoto,83.84 -Shiga,83.98 -Shiga,84.24 -Kumamoto,84.36 -Kumamoto,84.63 -Shiga,84.89 -Nagano,84.96 -Shiga,85.15 -Shiga,85.34 -Nagano,85.54 -Kyoto,85.6 -Shiga,85.09 -Shiga,85.19 -Shiga
max_3,83.68 -Fukui,83.74 -Fukui,83.8 -Fukui,84.2 -Shiga,84.35 -Nagano,84.53 -Kumamoto,84.74 -Kyoto,84.89 -Kyoto,85.08 -Kumamoto,85.22 -Nara,85.51 -Nagano,85.39 -Kyoto,84.68 -Okayama,85.09 -Nara
Japan,– 82.95 –,– 82.71 –,– 83.21 –,– 83.43 –,– 83.68 –,– 83.92 –,– 84.1 –,– 84.22 –,– 84.32 –,– 84.46 –,– 84.71 –,– 84.58 –,– 84.1 –,– 84.14 –
Okinawa,– 83.24 –,– 83.0 –,– 83.6 –,– 83.65 –,– 83.56 –,– 83.95 –,– 83.97 –,– 84.06 –,– 84.32 –,– 84.26 –,– 84.52 –,– 83.99 –,– 83.26 –,– 83.39 –
min_3,82.17 -Iwate,80.53 -Fukushima,82.59 -Iwate,82.62 -Akita,82.83 -Akita,83.18 -Iwate,83.29 -Iwate,83.35 -Iwate,83.47 -Fukushima,83.58 -Fukushima,83.96 -Fukushima,83.96 -Gumma,83.23 -Fukushima,83.23 -Akita
min_2,82.11 -Akita,75.17 -Iwate,82.32 -Tochigi,82.57 -Tokushima,82.82 -Tochigi,83.09 -Fukushima,83.11 -Akita,83.23 -Akita,83.41 -Iwate,83.5 -Iwate,83.93 -Ibaraki,83.59 -Fukushima,83.18 -Iwate,83.19 -Iwate
min,81.33 -Aomori,74.51 -Miyagi,81.5 -Aomori,81.91 -Aomori,82.05 -Aomori,82.41 -Aomori,82.65 -Aomori,82.74 -Aomori,82.81 -Aomori,82.6 -Aomori,83.09 -Aomori,82.96 -Aomori,82.69 -Aomori,82.22 -Aomori


In [10]:
# for male
mal.min_and_max_values(df_m, row_center=['Japan', 'Okinawa'], nmb=3, max_lng=9)

Number of records: 48


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
max,80.82 -Nagano,80.86 -Nagano,81.18 -Shiga,81.37 -Nagano,81.53 -Shiga,81.85 -Nagano,81.97 -Nagano,82.11 -Nagano,82.44 -Nagano,82.55 -Nagano,82.88 -Shiga,82.94 -Nagano,82.4 -Nagano,82.58 -Nagano
max_2,80.51 -Shiga,80.66 -Shiga,80.97 -Nagano,81.0 -Shiga,81.31 -Nagano,81.73 -Shiga,81.93 -Shiga,81.92 -Shiga,82.33 -Shiga,82.54 -Shiga,82.54 -Nagano,82.75 -Shiga,82.18 -Shiga,82.38 -Nara
max_3,80.4 -Fukui,80.54 -Nara,80.6 -Fukui,80.92 -Kumamoto,81.02 -Kyoto,81.44 -Nara,81.71 -Nara,81.86 -Kyoto,82.03 -Kyoto,82.26 -Nara,82.45 -Nara,82.45 -Nara,81.74 -Nara,82.31 -Shiga
Japan,– 79.51 –,– 79.41 –,– 79.91 –,– 80.17 –,– 80.46 –,– 80.72 –,– 80.94 –,– 81.06 –,– 81.22 –,– 81.37 –,– 81.58 –,– 81.49 –,– 81.06 –,– 81.1 –
Okinawa,– 79.34 –,– 79.04 –,– 79.75 –,– 79.97 –,– 79.92 –,– 80.35 –,– 80.3 –,– 80.35 –,– 81.01 –,– 80.66 –,– 80.92 –,– 80.5 –,– 79.91 –,– 79.94 –
min_3,78.45 -Iwate,77.29 -Fukushima,79.01 -Akita,79.0 -Tokushima,79.62 -Iwate,79.74 -Iwate,79.99 -Wakayama,79.85 -Iwate,80.02 -Akita,80.4 -Fukushima,80.87 -Fukushima,80.57 -Fukushima,80.01 -Iwate,79.92 -Akita
min_2,78.18 -Akita,72.3 -Iwate,78.89 -Iwate,78.84 -Akita,79.22 -Akita,79.57 -Akita,79.7 -Akita,79.71 -Akita,79.97 -Iwate,80.14 -Iwate,80.46 -Akita,80.5 -Okinawa,79.91 -Okinawa,79.68 -Iwate
min,77.21 -Aomori,72.22 -Miyagi,77.77 -Aomori,78.16 -Aomori,78.12 -Aomori,78.8 -Aomori,78.99 -Aomori,78.99 -Aomori,79.2 -Aomori,78.94 -Aomori,79.43 -Aomori,79.42 -Aomori,79.32 -Aomori,78.78 -Aomori


In [11]:
# for female
mal.min_and_max_values(df_f, row_center=['Japan', 'Okinawa'], nmb=3, max_lng=9)

Number of records: 48


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
max,87.12 -Nagano,87.06 -Shimane,87.38 -Okinawa,87.45 -Nagano,87.58 -Shimane,87.91 -Nagano,87.91 -Shiga,87.93 -Okayama,88.14 -Shimane,88.21 -Shimane,88.68 -Tottori,88.36 -Shiga,87.97 -Shiga,88.14 -Shimane
max_2,86.99 -Okinawa,87.04 -Fukui,87.06 -Shimane,87.26 -Shiga,87.5 -Fukui,87.59 -Shimane,87.8 -Okayama,87.9 -Shiga,88.12 -Fukui,88.12 -Okayama,88.53 -Okayama,88.33 -Shimane,87.96 -Shimane,88.06 -Shiga
max_3,86.98 -Shimane,86.93 -Okayama,86.99 -Niigata,87.25 -Okinawa,87.48 -Okayama,87.56 -Okayama,87.69 -Nagano,87.84 -Okinawa,88.04 -Tottori,88.03 -Shiga,88.51 -Kumamoto,88.31 -Nagano,87.91 -Nagano,87.96 -Nagano
Japan,– 86.24 –,– 85.86 –,– 86.36 –,– 86.56 –,– 86.77 –,– 86.97 –,– 87.12 –,– 87.26 –,– 87.31 –,– 87.44 –,– 87.73 –,– 87.6 –,– 87.11 –,– 87.16 –
Okinawa,– 86.99 –,– 86.9 –,– 87.38 –,– 87.25 –,– 87.18 –,– 87.53 –,– 87.67 –,– 87.84 –,– 87.63 –,– 87.92 –,– 88.17 –,– 87.6 –,– 86.78 –,– 87.07 –
min_3,85.62 -Wakayama,83.75 -Fukushima,85.85 -Ibaraki,85.92 -Wakayama,86.19 -Akita,86.33 -Ibaraki,86.37 -Tochigi,86.53 -Fukushima,86.61 -Fukushima,86.71 -Ibaraki,87.07 -Ibaraki,86.91 -Hokkaido,86.31 -Iwate,86.38 -Tochigi
min_2,85.57 -Tochigi,77.98 -Iwate,85.62 -Tochigi,85.85 -Tochigi,85.95 -Tochigi,86.26 -Fukushima,86.26 -Akita,86.36 -Tochigi,86.56 -Tochigi,86.6 -Tochigi,87.05 -Fukushima,86.63 -Fukushima,86.29 -Fukushima,86.22 -Fukushima
min,85.22 -Aomori,76.73 -Miyagi,84.99 -Aomori,85.44 -Aomori,85.77 -Aomori,85.74 -Aomori,86.04 -Aomori,86.26 -Aomori,86.21 -Aomori,86.09 -Aomori,86.58 -Aomori,86.33 -Aomori,85.89 -Aomori,85.55 -Aomori


<br>

In [13]:
dd_replacement = {
    'Japan'     : {'jp': ('日本', ''), 'en': ('Japan', ''), 'ru': ('Япония', '')},
    'Hokkaido'  : {'jp': ('北海道', ''), 'en': ('Hokkaido', 'Hokkaido'), 'ru': ('Хокка́йдо', 'Хоккайдо')},
    'Aomori'    : {'jp': ('青森県', ''), 'en': ('Aomori ', 'Aomori Prefecture'), 'ru': ('Ао́мори', 'Аомори (префектура)')},
    'Iwate'     : {'jp': ('岩手県', ''), 'en': ('Iwate', 'Iwate Prefecture'), 'ru': ('Ива́те', 'Ивате (префектура)')},
    'Miyagi'    : {'jp': ('宮城県', ''), 'en': ('Miyagi', 'Miyagi Prefecture'), 'ru': ('Мия́ги', 'Мияги (префектура)')},
    'Akita'     : {'jp': ('秋田県', ''), 'en': ('Akita', 'Akita Prefecture'), 'ru': ('А́кита', 'Акита (префектура)')},
    'Yamagata'  : {'jp': ('山形県', ''), 'en': ('Yamagata', 'Yamagata Prefecture'), 'ru': ('Ямага́та', 'Ямагата (префектура)')},
    'Fukushima' : {'jp': ('福島県', ''), 'en': ('Fukushima', 'Fukushima Prefecture'), 'ru': ('Фуку́сима', 'Фукусима (префектура)')},
    'Ibaraki'   : {'jp': ('茨城県', ''), 'en': ('Ibaraki', 'Ibaraki Prefecture'), 'ru': ('Ибара́ки', 'Ибараки')},
    'Tochigi'   : {'jp': ('栃木県', ''), 'en': ('Tochigi', 'Tochigi Prefecture'), 'ru': ('Тоти́ги', 'Тотиги (префектура)')},
    'Gumma'     : {'jp': ('群馬県', ''), 'en': ('Gunma', 'Gunma Prefecture'), 'ru': ('Гумма', 'Гумма (префектура)')},
    'Saitama'   : {'jp': ('埼玉県', ''), 'en': ('Saitama', 'Saitama Prefecture'), 'ru': ('Са́йтама', 'Сайтама (префектура)')},
    'Chiba'     : {'jp': ('千葉県', ''), 'en': ('Chiba', 'Chiba Prefecture'), 'ru': ('Тиба', 'Тиба (префектура)')},
    'Tokyo'     : {'jp': ('東京都', ''), 'en': ('Tokyo', 'Tokyo'), 'ru': ('То́кио', 'Токио')},
    'Kanagawa'  : {'jp': ('神奈川県', ''), 'en': ('Kanagawa', 'Kanagawa Prefecture'), 'ru': ('Канага́ва', 'Канагава')},
    'Niigata'   : {'jp': ('新潟県', ''), 'en': ('Niigata', 'Niigata Prefecture'), 'ru': ('Ниига́та', 'Ниигата (префектура)')},
    'Toyama'    : {'jp': ('富山県', ''), 'en': ('Toyama', 'Toyama Prefecture'), 'ru': ('Тоя́ма', 'Тояма (префектура)')},
    'Ishikawa'  : {'jp': ('石川県', ''), 'en': ('Ishikawa', 'Ishikawa Prefecture'), 'ru': ('Исика́ва', 'Исикава')},
    'Fukui'     : {'jp': ('福井県', ''), 'en': ('Fukui', 'Fukui Prefecture'), 'ru': ('Фукуи́', 'Фукуи (префектура)')},
    'Yamanashi' : {'jp': ('山梨県', ''), 'en': ('Yamanashi', 'Yamanashi Prefecture'), 'ru': ('Ямана́си', 'Яманаси (префектура)')},
    'Nagano'    : {'jp': ('長野県', ''), 'en': ('Nagano', 'Nagano Prefecture'), 'ru': ('Нага́но', 'Нагано (префектура)')},
    'Gifu'      : {'jp': ('岐阜県', ''), 'en': ('Gifu', 'Gifu Prefecture'), 'ru': ('Ги́фу', 'Гифу (префектура)')},
    'Shizuoka'  : {'jp': ('静岡県', ''), 'en': ('Shizuoka', 'Shizuoka Prefecture'), 'ru': ('Сидзуо́ка', 'Сидзуока (префектура)')},
    'Aichi'     : {'jp': ('愛知県', ''), 'en': ('Aichi', 'Aichi Prefecture'), 'ru': ('А́йти', 'Айти (префектура)')},
    'Mie'       : {'jp': ('三重県', ''), 'en': ('Mie', 'Mie Prefecture'), 'ru': ('Ми́э', 'Миэ')},
    'Shiga'     : {'jp': ('滋賀県', ''), 'en': ('Shiga', 'Shiga Prefecture'), 'ru': ('Си́га', 'Сига')},
    'Kyoto'     : {'jp': ('都府', ''), 'en': ('Kyoto', 'Kyoto Prefecture'), 'ru': ('Кио́то', 'Киото (префектура)')},
    'Osaka'     : {'jp': ('大阪府', ''), 'en': ('Osaka', 'Osaka Prefecture'), 'ru': ('О́сака', 'Осака (префектура)')},
    'Hyogo'     : {'jp': ('兵庫県', ''), 'en': ('Hyōgo', 'Hyōgo Prefecture'), 'ru': ('Хио́го', 'Хиого')},
    'Nara'      : {'jp': ('奈良県', ''), 'en': ('Nara', 'Nara Prefecture'), 'ru': ('На́ра', 'Нара (префектура)')},
    'Wakayama'  : {'jp': ('和歌山県', ''), 'en': ('Wakayama', 'Wakayama Prefecture'), 'ru': ('Вакая́ма', 'Вакаяма (префектура)')},
    'Tottori'   : {'jp': ('鳥取県', ''), 'en': ('Tottori', 'Tottori Prefecture'), 'ru': ('Тотто́ри', 'Тоттори (префектура)')},
    'Shimane'   : {'jp': ('島根県', ''), 'en': ('Shimane', 'Shimane Prefecture'), 'ru': ('Сима́не', 'Симане')},
    'Okayama'   : {'jp': ('岡山県', ''), 'en': ('Okayama', 'Okayama Prefecture'), 'ru': ('Окаяма', 'Окаяма (префектура)')},
    'Hiroshima' : {'jp': ('広島県', ''), 'en': ('Hiroshima', 'Hiroshima Prefecture'), 'ru': ('Хироси́ма', 'Хиросима (префектура)')},
    'Yamaguchi' : {'jp': ('山口県', ''), 'en': ('Yamaguchi', 'Yamaguchi Prefecture'), 'ru': ('Ямагу́ти', 'Ямагути (префектура)')},
    'Tokushima' : {'jp': ('徳島県', ''), 'en': ('Tokushima', 'Tokushima Prefecture'), 'ru': ('Токуси́ма', 'Токусима (префектура)')},
    'Kagawa'    : {'jp': ('香川県', ''), 'en': ('Kagawa', 'Kagawa Prefecture'), 'ru': ('Кагава', 'Кагава (префектура)')},
    'Ehime'     : {'jp': ('愛媛県', ''), 'en': ('Ehime', 'Ehime Prefecture'), 'ru': ('Эхиме́', 'Эхиме')},
    'Kochi'     : {'jp': ('高知県', ''), 'en': ('Kōchi', 'Kōchi Prefecture'), 'ru': ('Ко́ти', 'Коти (префектура)')},
    'Fukuoka'   : {'jp': ('福岡県', ''), 'en': ('Fukuoka', 'Fukuoka Prefecture'), 'ru': ('Фукуо́ка', 'Фукуока (префектура)')},
    'Saga'      : {'jp': ('佐賀県', ''), 'en': ('Saga', 'Saga Prefecture'), 'ru': ('Са́га', 'Сага (префектура)')},
    'Nagasaki'  : {'jp': ('長崎県', ''), 'en': ('Nagasaki', 'Nagasaki Prefecture'), 'ru': ('Нагаса́ки', 'Нагасаки (префектура)')},
    'Kumamoto'  : {'jp': ('熊本県', ''), 'en': ('Kumamoto', 'Kumamoto Prefecture'), 'ru': ('Кумамо́то', 'Кумамото (префектура)')},
    'Oita'      : {'jp': ('大分県', ''), 'en': ('Ōita', 'Ōita Prefecture'), 'ru': ('О́ита', 'Оита (префектура)')},
    'Miyazaki'  : {'jp': ('宮崎県', ''), 'en': ('Miyazaki', 'Miyazaki Prefecture'), 'ru': ('Миядза́ки', 'Миядзаки (префектура)')},
    'Kagoshima' : {'jp': ('鹿児島県', ''), 'en': ('Kagoshima', 'Kagoshima Prefecture'), 'ru': ('Каго́сима', 'Кагосима (префектура)')},
    'Okinawa'   : {'jp': ('沖縄県', ''), 'en': ('Okinawa', 'Okinawa Prefecture'), 'ru': ('Окина́ва', 'Окинава (префектура)')}
}

In [14]:
# create code for placing info in Wikipedia
def create_table_v1(df_t, df_m, df_f, file_header, lang='en'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
    def chval(x, prec=2, *, add_par=''):  # change_value
        return f'style="{add_par}"| —' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;{add_par}"| {x:0.{prec}f}' if x>0 else \
               f'style="background:#fffae0;color:crimson;{add_par}"| −{-x:0.{prec}f}' if x<0 else \
               f'style="background:#fffae0;color:darkgray;{add_par}"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=2, *, add_par=''):  # change_value
        return ' —' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;{add_par}"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="background:#fffae0;color:crimson;{add_par}"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="background:#fffae0;color:darkgray;{add_par}"| \'\'\'{x:0.{prec}f}\'\'\''
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()

    st = ''
    for i in range(len(df_t)):
        ser_t = df_t.iloc[i]
        ser_m = df_m.iloc[i]
        ser_f = df_f.iloc[i]
        if ser_t.name == 'Japan':
             st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{dd_replacement[ser_t.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser_m[2020])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser_f[2020])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser_f[2020] - ser_m[2020])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser_t[2013])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2018] - ser_t[2013], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2018])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2019] - ser_t[2018], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2019])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2020] - ser_t[2019], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2021] - ser_t[2020], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2021])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2022] - ser_t[2021], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2022])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2023] - ser_t[2022], add_par="padding-right:1.5ex;")} ' + \
                  f'||style="border-left-width:2px;padding-right:1.5ex;background:#e0ffd8;"| \'\'\'{if_value(ser_t[2023])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser_m[2023])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser_f[2023])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser_f[2023] - ser_m[2023])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2023] - ser_t[2013], add_par="border-left-width:2px;padding-right:1.5ex;")}'
        else:
            name_link = dd_replacement[ser_t.name][lang][1]
            name_visible = dd_replacement[ser_t.name][lang][0]
            name_inserted = name_link if name_link == name_visible else f"{name_link}|{name_visible}"
            st += '\n' + '|-\n' + \
                  f'| [[{name_inserted}]] ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser_m[2020])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser_f[2020])} ' + \
                  f'|| {if_value(ser_f[2020] - ser_m[2020])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser_t[2013])} ' + \
                  f'||{chval(ser_t[2018] - ser_t[2013], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2018])} ' + \
                  f'||{chval(ser_t[2019] - ser_t[2018], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2019])} ' + \
                  f'||{chval(ser_t[2020] - ser_t[2019], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||{chval(ser_t[2021] - ser_t[2020], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2021])} ' + \
                  f'||{chval(ser_t[2022] - ser_t[2021], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2022])} ' + \
                  f'||{chval(ser_t[2023] - ser_t[2022], add_par="padding-right:1.5ex;")} ' + \
                  f'||style="border-left-width:2px;padding-right:1.5ex;background:#e0ffd8;"| \'\'\'{if_value(ser_t[2023])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser_m[2023])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser_f[2023])} ' + \
                  f'|| {if_value(ser_f[2023] - ser_m[2023])} ' + \
                  f'||{chval(ser_t[2023] - ser_t[2013], add_par="border-left-width:2px;padding-right:1.5ex;")}'

    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        st = st.replace('padding-right:1,5ex;', 'padding-right:1.5ex;')
        
    st = table_header + st + '\n|}'
    
    # gray color for missing values
    st = st.replace(';"|—', ';color:silver;"|—')

    return st


table_code = create_table_v1(df_t, df_m, df_f, file_header='Japanese_header_en -2023 -v1.txt', lang='en')
with open('output/Table code for Japanese prefectures -en -v1.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

table_code = create_table_v1(df_t, df_m, df_f, file_header='Japanese_header_ru -2023 -v1.txt', lang='ru')
with open('output/Table code for Japanese prefectures -ru -v1.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br>

In [16]:
# create code for placing info in Wikipedia
def create_table_v2(df_t, df_m, df_f, file_header, lang='en'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
    def chval(x, prec=2, *, add_par=''):  # change_value
        return f'style="{add_par}"| —' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;{add_par}"| {x:0.{prec}f}' if x>0 else \
               f'style="background:#fffae0;color:crimson;{add_par}"| −{-x:0.{prec}f}' if x<0 else \
               f'style="background:#fffae0;color:darkgray;{add_par}"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=2, *, add_par=''):  # change_value
        return ' —' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;{add_par}"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="background:#fffae0;color:crimson;{add_par}"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="background:#fffae0;color:darkgray;{add_par}"| \'\'\'{x:0.{prec}f}\'\'\''
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()

    st = ''
    for i in range(len(df_t)):
        ser_t = df_t.iloc[i]
        ser_m = df_m.iloc[i]
        ser_f = df_f.iloc[i]
        if ser_t.name == 'Japan':
             st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{dd_replacement[ser_t.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser_m[2020])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser_f[2020])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser_f[2020] - ser_m[2020])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser_t[2010])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2015] - ser_t[2010], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2015])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2019] - ser_t[2015], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2019])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2020] - ser_t[2019], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2021] - ser_t[2020], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2021])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2022] - ser_t[2021], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2022])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2023] - ser_t[2022], add_par="padding-right:1.5ex;")} ' + \
                  f'||style="border-left-width:2px;padding-right:1.5ex;background:#e0ffd8;"| \'\'\'{if_value(ser_t[2023])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser_m[2023])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser_f[2023])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser_f[2023] - ser_m[2023])}\'\'\' ' + \
                  f'||{chval_bold(ser_t[2020] - ser_t[2010], add_par="border-left-width:2px;padding-right:1.5ex;")} ' + \
                  f'||{chval_bold(ser_t[2023] - ser_t[2020], add_par="padding-right:1.5ex;")} ' + \
                  f'||{chval_bold(ser_t[2023] - ser_t[2010], add_par="padding-right:1.5ex;")}'
            
        else:
            name_link = dd_replacement[ser_t.name][lang][1]
            name_visible = dd_replacement[ser_t.name][lang][0]
            name_inserted = name_link if name_link == name_visible else f"{name_link}|{name_visible}"
            st += '\n' + '|-\n' + \
                  f'| [[{name_inserted}]] ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser_m[2020])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser_f[2020])} ' + \
                  f'|| {if_value(ser_f[2020] - ser_m[2020])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser_t[2010])} ' + \
                  f'||{chval(ser_t[2015] - ser_t[2010], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2015])} ' + \
                  f'||{chval(ser_t[2019] - ser_t[2015], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2019])} ' + \
                  f'||{chval(ser_t[2020] - ser_t[2019], add_par="padding-right:1.5ex;")} ' + \
                  f'|| \'\'\'{if_value(ser_t[2020])}\'\'\' ' + \
                  f'||{chval(ser_t[2021] - ser_t[2020], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2021])} ' + \
                  f'||{chval(ser_t[2022] - ser_t[2021], add_par="padding-right:1.5ex;")} ' + \
                  f'|| {if_value(ser_t[2022])} ' + \
                  f'||{chval(ser_t[2023] - ser_t[2022], add_par="padding-right:1.5ex;")} ' + \
                  f'||style="border-left-width:2px;padding-right:1.5ex;background:#e0ffd8;"| \'\'\'{if_value(ser_t[2023])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser_m[2023])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser_f[2023])} ' + \
                  f'|| {if_value(ser_f[2023] - ser_m[2023])} ' + \
                  f'||{chval(ser_t[2020] - ser_t[2010], add_par="border-left-width:2px;padding-right:1.5ex;")} ' + \
                  f'||{chval(ser_t[2023] - ser_t[2020], add_par="padding-right:1.5ex;")} ' + \
                  f'||{chval(ser_t[2023] - ser_t[2010], add_par="padding-right:1.5ex;")}'

    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        st = st.replace('padding-right:1,5ex;', 'padding-right:1.5ex;')
        
    st = table_header + st + '\n|}'
    
    # gray color for missing values
    st = st.replace(';"|—', ';color:silver;"|—')

    return st


table_code = create_table_v2(df_t, df_m, df_f, file_header='Japanese_header_en -2023 -v2.txt', lang='en')
with open('output/Table code for Japanese prefectures -en -v2.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

table_code = create_table_v2(df_t, df_m, df_f, file_header='Japanese_header_ru -2023 -v2.txt', lang='ru')
with open('output/Table code for Japanese prefectures -ru -v2.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)